In [1]:
import logging
import os
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=10,
    seed=0,
)

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 3
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [3]:
t_c = 100
t_v = 100
K = 2
n = 2
max_iter = 2
solver_class = TSP_LKH_Solver
instance_class = TSP_Instance

experiment = PcitExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-17 16:19:59] INFO      [PCIT] Start!
[2024-11-17 16:19:59] INFO      Attempt 1/2
[2024-11-17 16:19:59] INFO      Phase 1/2
[2024-11-17 16:19:59] INFO      Clustering: {'data/TSP/CEPS_benchmark/cluster_netgen/40.tsp': 0, 'data/TSP/CEPS_benchmark/grid/04.tsp': 1, 'data/TSP/CEPS_benchmark/grid/33.tsp': 0, 'data/TSP/CEPS_benchmark/rotation/45.tsp': 1, 'data/TSP/CEPS_benchmark/uniform_portgen/11.tsp': 0, 'data/TSP/CEPS_benchmark/cluster/15.tsp': 1, 'data/TSP/CEPS_benchmark/implosion/16.tsp': 0, 'data/TSP/CEPS_benchmark/uniform_portgen/39.tsp': 1, 'data/TSP/CEPS_benchmark/expansion/09.tsp': 1, 'data/TSP/CEPS_benchmark/expansion/03.tsp': 0}
[2024-11-17 16:19:59] DEBUG     =================================  Portfolio  ==================================
[2024-11-17 16:19:59] DEBUG     2220588739511820280 --> ASCENT_CANDIDATES=52;BACKBONE_TRIALS=1;BACKTRACKING=YES;CANDIDATE_SET_TYPE=ALPHA;EXTRA_CANDIDATES=7;EXTRA_CANDIDATE_SET_TYPE=QUADRANT;GAIN23=NO;GAIN_CRITERION=YES;INITIAL_STEP_SIZE

KeyboardInterrupt: 

In [ ]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [ ]:
from src.database import db_connect
conn = db_connect()

In [ ]:
import sqlite3
# conn = sqlite3.connect("_archive/3/PARHYDRA.db")
conn = sqlite3.connect("database/2024_11_16_23_14_00.db")

In [ ]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

In [ ]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

In [ ]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

In [ ]:
df.loc[df["solver_id"] == 1333412370196243425]

In [ ]:
574052600841816340

In [ ]:
df.loc[df["comment"] == "configuration"]

In [ ]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

In [ ]:
_["min"].describe()

In [ ]:
(_["min"] > 90).value_counts()

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index